# Capstone Project

In [61]:
import pandas as pd
import numpy as np

In [62]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [63]:
from bs4 import BeautifulSoup
import requests

In [64]:
soup=BeautifulSoup(requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text,'html.parser')
#print(soup.prettify())

### Extract  table data to list

In [65]:
data = []
table = soup.find('table', attrs={'class':'wikitable sortable'})
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) # Get rid of empty values
data

[[],
 ['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront'],
 ['M5A', 'Downtown Toronto', 'Regent Park'],
 ['M6A', 'North York', 'Lawrence Heights'],
 ['M6A', 'North York', 'Lawrence Manor'],
 ['M7A', "Queen's Park", 'Not assigned'],
 ['M8A', 'Not assigned', 'Not assigned'],
 ['M9A', 'Etobicoke', 'Islington Avenue'],
 ['M1B', 'Scarborough', 'Rouge'],
 ['M1B', 'Scarborough', 'Malvern'],
 ['M2B', 'Not assigned', 'Not assigned'],
 ['M3B', 'North York', 'Don Mills North'],
 ['M4B', 'East York', 'Woodbine Gardens'],
 ['M4B', 'East York', 'Parkview Hill'],
 ['M5B', 'Downtown Toronto', 'Ryerson'],
 ['M5B', 'Downtown Toronto', 'Garden District'],
 ['M6B', 'North York', 'Glencairn'],
 ['M7B', 'Not assigned', 'Not assigned'],
 ['M8B', 'Not assigned', 'Not assigned'],
 ['M9B', 'Etobicoke', 'Cloverdale'],
 ['M9B', 'Etobicoke', 'Islington'],
 ['M

## Data list to DataFrame

In [82]:
df=pd.DataFrame(columns=['PostalCode','Borough','Neighborhood'])
for row in data:
    if row:
        if row[1]!='Not assigned':
            neighborhood=row[2] if row[2]!='Not assigned' else row[1]
            if row[0] in df['PostalCode'].values:
                n_set=set(df[df['PostalCode']==row[0]].iloc[0]['Neighborhood'].split(','))
                n_set.add(neighborhood)
                neighborhood=','.join(n_set)
                df.at[df.index[df['PostalCode']==row[0]],'Neighborhood']=neighborhood
            else:
                df=df.append({'PostalCode':row[0],
                           'Borough':row[1],
                           'Neighborhood':neighborhood
                },ignore_index=True)
            
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park,Harbourfront"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [83]:
df.shape

(103, 3)

## Lat and Lot added to DataFrame

In [84]:
df_geocode=pd.read_csv('Geospatial_Coordinates.csv')
print(df_geocode.shape)
df_geocode.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [85]:
import geocoder # import geocoder

ll=pd.DataFrame(columns=['Latitude','Longitude'])
# loop until you get the coordinates
for postal_code in df['PostalCode']:
    latlon=df_geocode[df_geocode['Postal Code']==postal_code]
    ll=ll.append({'Latitude':latlon.iloc[0]['Latitude'],
                  'Longitude':latlon.iloc[0]['Longitude']
    },ignore_index=True)
    
ll.head()

,Latitude,Longitude
0,43.753259,-79.329656
1,43.725882,-79.315572
2,43.654260,-79.360636
3,43.718518,-79.464763
4,43.662301,-79.389494


In [86]:
df_concat=pd.concat([df,ll],axis=1).reset_index(drop=True)
df_concat.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [87]:
df_concat.shape

(103, 5)

In [92]:
df_toronto=df_concat[df_concat.Borough.str.contains('Toronto')].reset_index(drop=True)
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District,Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [93]:
df_toronto.shape

(38, 5)

In [94]:
CLIENT_ID = 'HOFZJ1KBNJRXXBLO1XLXSZLBNPGVW11G5MO551W3W1O1FJSW' # your Foursquare ID
CLIENT_SECRET = 'RTPC5KJ2U5JWPLRF2RH1X5VP20FRJWDMRWZVQHZQTSQPZOR4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 125
RADIUS = 700

In [95]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [99]:
toronto_venues = getNearbyVenues(
    names=df_toronto['PostalCode'],
    latitudes=df_toronto['Latitude'],
    longitudes=df_toronto['Longitude']
)
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M5A,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,M5A,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,M5A,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [100]:
toronto_venues.shape

(1705, 7)

In [101]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
M4E,5,5,5,5,5,5
M4K,41,41,41,41,41,41
M4L,17,17,17,17,17,17
M4M,41,41,41,41,41,41
M4N,3,3,3,3,3,3
M4P,9,9,9,9,9,9
M4R,20,20,20,20,20,20
M4S,33,33,33,33,33,33
M4T,3,3,3,3,3,3


In [102]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))


There are 231 uniques categories.


### Part 3 Explore and cluster the neighborhoods in Toronto
This is pretty much copy and paste from the lab

#### 3. Analyze Each Neighborhood
First, build a dataframe where each venue is one-hot encoded so the columns are the venue category values, and the data contains 1 under the column of it's category

I had to change the first column from 'Neighborhood' back to 'Postcode', because there is a category with the value 'Neighborhood'

So now we end up with 234 columns -- one column for the Postcode of a venue, and 233 columns for each possible venue category.

In [103]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column (as its original title Postcode) back to dataframe
# (This is not the same way it was done in the lab, but it's simpler code)
toronto_onehot.insert(loc=0, column='Postcode', value=toronto_venues['Neighborhood'])

toronto_onehot.head()

,Postcode,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [104]:
toronto_grouped = toronto_onehot.groupby('Postcode').mean().reset_index()
toronto_grouped

,Postcode,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M4E,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.20000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,M4K,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.024390,0.000000,...,0.000000,0.02439,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024390
2,M4L,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,M4M,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.048780,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024390
4,M4N,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,M4P,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,M4R,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.050000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000
7,M4S,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.030303,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,M4T,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,M4V,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.066667,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000


In [105]:
toronto_grouped.shape


(38, 232)

In [ ]:
num_top_venues = 5

for hood in toronto_grouped['Postcode']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped.Postcode == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [113]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [114]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['Postcode'] = toronto_grouped['Postcode']

for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_venues_sorted.head()

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Health Food Store,Other Great Outdoors,Neighborhood,Pub,Trail,Event Space,Ethiopian Restaurant,Electronics Store,Discount Store,Falafel Restaurant
1,M4K,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Yoga Studio,Pizza Place,Bookstore,Brewery,Bubble Tea Shop
2,M4L,Brewery,Fast Food Restaurant,Sushi Restaurant,Italian Restaurant,Sandwich Place,Movie Theater,Pub,Ice Cream Shop,Fish & Chips Shop,Steakhouse
3,M4M,Café,Coffee Shop,Italian Restaurant,Bakery,American Restaurant,Gastropub,Cheese Shop,Bookstore,Fish Market,Latin American Restaurant
4,M4N,Park,Swim School,Bus Line,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


### 4. Cluster Neighborhoods
Run k-means to cluster the neighborhood into 5 clusters.

In [115]:
from sklearn.cluster import KMeans


In [120]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Postcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=44).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([9, 3, 3, 0, 5, 4, 3, 3, 7, 3, 2, 3, 3, 0, 3, 0, 0, 0, 0, 0, 0, 0,
       1, 6, 0, 0, 3, 8, 0, 0, 0, 3, 3, 0, 3, 3, 3, 3])

In [123]:
# add clustering labels
toronto_venues_sorted['Cluster Labels'] = kmeans.labels_
#toronto_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_venues_sorted.set_index('Postcode'), on='PostalCode')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.654260,-79.360636,Coffee Shop,Café,Park,Bakery,Restaurant,Mexican Restaurant,Theater,Pub,Breakfast Spot,Gym / Fitness Center,0
1,M5B,Downtown Toronto,"Garden District,Ryerson",43.657162,-79.378937,Coffee Shop,Clothing Store,Cosmetics Shop,Fast Food Restaurant,Café,Theater,Tea Room,Bookstore,Italian Restaurant,Japanese Restaurant,3
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Gastropub,Cocktail Bar,Bakery,Cosmetics Shop,Breakfast Spot,0
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Health Food Store,Other Great Outdoors,Neighborhood,Pub,Trail,Event Space,Ethiopian Restaurant,Electronics Store,Discount Store,Falafel Restaurant,9
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,Coffee Shop,Cocktail Bar,Beer Bar,Café,Steakhouse,Bakery,Seafood Restaurant,Farmers Market,Cheese Shop,Belgian Restaurant,0


In [124]:
toronto_merged[toronto_merged['Cluster Labels'] != 9]


,PostalCode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.654260,-79.360636,Coffee Shop,Café,Park,Bakery,Restaurant,Mexican Restaurant,Theater,Pub,Breakfast Spot,Gym / Fitness Center,0
1,M5B,Downtown Toronto,"Garden District,Ryerson",43.657162,-79.378937,Coffee Shop,Clothing Store,Cosmetics Shop,Fast Food Restaurant,Café,Theater,Tea Room,Bookstore,Italian Restaurant,Japanese Restaurant,3
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Gastropub,Cocktail Bar,Bakery,Cosmetics Shop,Breakfast Spot,0
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,Coffee Shop,Cocktail Bar,Beer Bar,Café,Steakhouse,Bakery,Seafood Restaurant,Farmers Market,Cheese Shop,Belgian Restaurant,0
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Ice Cream Shop,Burger Joint,Bubble Tea Shop,Spa,Sushi Restaurant,Bar,0
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564,Grocery Store,Café,Park,Diner,Convenience Store,Baby Store,Italian Restaurant,Coffee Shop,Restaurant,Nightclub,0
7,M5H,Downtown Toronto,"King,Adelaide,Richmond",43.650571,-79.384568,Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,Restaurant,Hotel,Cosmetics Shop,Burger Joint,American Restaurant,0
8,M6H,West Toronto,"Dovercourt Village,Dufferin",43.669005,-79.442259,Bakery,Pharmacy,Supermarket,Gym / Fitness Center,Music Venue,Discount Store,Middle Eastern Restaurant,Café,Brewery,Fast Food Restaurant,3
9,M5J,Downtown Toronto,"Union Station,Harbourfront East,Toronto Islands",43.640816,-79.381752,Coffee Shop,Aquarium,Hotel,Café,Italian Restaurant,Scenic Lookout,Brewery,Bakery,Sporting Goods Shop,Pizza Place,0
10,M6J,West Toronto,"Trinity,Little Portugal",43.647927,-79.419750,Bar,Coffee Shop,Asian Restaurant,Boutique,Café,Restaurant,Pizza Place,Cocktail Bar,Bakery,French Restaurant,3


In [125]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from IPython.display import Markdown, display, HTML

In [127]:
# create map
latitude = 43.6532
longitude = -79.3832
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

# HTML(m._repr_html_())
HTML(map_clusters._repr_html_())